In [1]:
# Imports
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

# Hyperparams
# training config
NUM_EPOCHS= 5
LR=0.001
# dataset config
batch_size = 128
generator=torch.Generator().manual_seed(42) # Can be included for reproducability

In [2]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [3]:
_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

NUM_CLASSES = 0

def getTrainingSet(dataset_name):
  if dataset_name == 'CIFAR-10':
    print("in cifar-10")
    NUM_CLASSES=10

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform_train)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                          download=True, transform=transform_test)
    

    trainset, validset = torch.utils.data.random_split(trainset, 
                                                      [int(len(trainset)*0.8),len(trainset)- 
                                                      int(len(trainset)*0.8)], generator=generator)
    
  elif dataset_name == 'STL10':
    NUM_CLASSES=10
    
    trainset = torchvision.datasets.STL10(root='./data', split='train',
                                          download=True, transform=transform_train)

    testset = torchvision.datasets.STL10(root='./data', split='test',
                                          download=True, transform=transform_train)
    

    trainset, validset = torch.utils.data.random_split(trainset, 
                                                      [int(len(trainset)*0.8),len(trainset)- 
                                                      int(len(trainset)*0.8)], generator=generator)
    
  elif dataset_name == 'tiny-imagenet':
    NUM_CLASSES=200
    
    #!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
    #!unzip -qq 'tiny-imagenet-200.zip'
    
    
    totalset = torchvision.datasets.ImageFolder('tiny-imagenet-200/train', 
                                                   transform=transform_train)
  
    train_counts = [0] * 200
    valid_counts = [0] * 200
    trainset = []
    validset = []
    testset = []
    for item in totalset:
        if train_counts[item[1]] < 350:
            trainset.append(item)
            train_counts[item[1]] += 1
        elif valid_counts[item[1]] < 75:
            validset.append(item)
            valid_counts[item[1]] += 1
        else:
            testset.append(item)

  elif dataset_name == 'Caltech101':
    NUM_CLASSES=101
    #!gdown https://drive.google.com/uc?id=1DX_XeKHn3yXtZ18DD7qc1wf-Jy5lnhD5
    #!unzip -qq '101_ObjectCategories.zip' 

    PATH = '101_ObjectCategories/'

    transform = transforms.Compose(
      [transforms.CenterCrop(256),
      transforms.Resize((64,64)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    totalset = torchvision.datasets.ImageFolder(PATH, transform=transform_train)

    X, y = zip(*totalset)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, 
                                                      stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, 
                                                    test_size = 0.5, 
                                                    stratify=y_val)

    trainset, validset, testset = list(zip(X_train, y_train)), list(zip(X_val, y_val)), list(zip(X_test, y_test))




  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                            shuffle=False,num_workers=2)
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
  return trainset, testset, trainloader, testloader

In [4]:
class VGGStyleNet(nn.Module):
    def __init__(self, num_classes: int = 10, init_weights: bool = True):
        super(VGGStyleNet, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(2)
        self.classifier = nn.Sequential(
            nn.Linear(128*8*8, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()
    
    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
        
    def forward(self, x):
        out = F.relu(self.conv1_1(x))
        out = F.relu(self.conv1_2(out))
        out = self.maxpool1(out)
        out = F.relu(self.conv2_1(out))
        out = F.relu(self.conv2_2(out))
        out = self.maxpool2(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [7]:
def train(epoch):
  net.train()
  correct_images = 0
  total_images = 0
  training_loss = 0
  losses = 0
  for batch_index, (images, labels) in enumerate(tqdm(trainloader)):
    optimizer.zero_grad()
    images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    training_loss += loss.item()
    _, predicted = outputs.max(1)
    total_images += labels.size(0)
    correct_images += predicted.eq(labels).sum().item()
  print('Epoch: %d, Loss: %.3f, '
              'Accuracy: %.3f%% (%d/%d)' % (epoch, training_loss/(batch_index+1),
                                       100.*correct_images/total_images, correct_images, total_images))
  return training_loss/(batch_index+1)

In [8]:
def test():
    test_loss = 0
    total_images = 0
    correct_images = 0
    total_loss = 0
    net.eval()
    with torch.no_grad():
      for batch_index, (images, labels) in enumerate(tqdm(testloader)):
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total_images += labels.size(0)
        correct_images += predicted.eq(labels).sum().item()
#         print(batch_index, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                   % (test_loss/(batch_index+1), 100.*correct_images/total_images, correct_images, total_images))
    test_accuracy = 100.*correct_images/total_images
    print("accuracy of test set is",test_accuracy)
    return test_accuracy

In [ ]:
#Code for paper one implementation

In [9]:
NUM_EPOCHS= 10
LR = 0.01
# dataset config
BATCH_SIZE = 32
dataset = 'CIFAR-10'
NUM_CLASSES = 10
full_batch = 20000

if dataset == 'Caltech101':
    NUM_CLASSES = 101
    full_batch = 6073
    #test size = 
elif dataset == 'STL10':
    NUM_CLASSES = 10
    full_batch = 4000
    #test size == 
elif dataset == 'CIFAR-10':
    NUM_CLASSES = 10
    full_batch = 20000
    #test size == 




In [14]:
optimizers = ['GD','SGD','ASGD']
loss_opts = []
accuracy_test = []
for opt in optimizers:
    loss = []
    
    batch_size = BATCH_SIZE
    LR = 0.01
    if opt == 'GD':
        batch_size = full_batch
        LR = 0.1
        
    print(batch_size)
    print(LR)
    
        
    # Model
    net = VGGStyleNet(NUM_CLASSES)
    net.to(device)
    trainset, testset, trainloader, testloader = getTrainingSet(dataset)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=LR)
    if opt == 'ASGD':
        optimizer = torch.optim.ASGD(net.parameters(), lr=LR)
    
    
    #RETURN LOSS AFTER EACH EPOCH
    for epoch in range(NUM_EPOCHS):
        loss.append(train(epoch))
        
    #RETURN ACCURACY    
    accuracy_test.append(test())
    
    loss_opts.append(loss)
    
print(loss_opts)
print(accuracy_test)

20000
0.1
in cifar-10


KeyboardInterrupt: 

In [13]:
#plot of data

import matplotlib.pyplot as plt
import numpy as np



plt.plot( range(1,21), loss_opts[2], marker='x', markerfacecolor='red', markersize=12, color='red', linewidth=4, label='GD')
plt.plot(range(1,21), loss_opts[0], marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4, label='SGD')
plt.plot( range(1,21), loss_opts[1], marker='x', markerfacecolor='green', markersize=12, color='olive', linewidth=4, linestyle='dashed', label='ASGD')

# show legend
plt.legend()

# show graph
plt.show()


print(accuracy_test)

IndexError: list index out of range